In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib
pd.set_option('display.max_columns', None)
from six.moves import reduce
import scipy
from sklearn.metrics import r2_score
#scipy.stats.pearsonr
import scipy.stats as stats
plt.rcParams.update({'font.family': 'Arial'})
import warnings
warnings.filterwarnings("ignore")
import math
import statistics

In [2]:
param_2018 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2018/PSO_2018_6params_NYC_norm_28_PSO_15.csv')
table_2018 = pd.read_csv('data/2_data_for_PSO_calibration/table_2018.csv')

param_2019 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2019/PSO_2019_6params_NYC_norm_28_PSO_15.csv')
table_2019 = pd.read_csv('data/2_data_for_PSO_calibration/table_2019.csv')

param_2020 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2020/PSO_2020_6params_NYC_norm_28_PSO_15.csv')
table_2020 = pd.read_csv('data/2_data_for_PSO_calibration/table_2020.csv')

param_2021 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2021/PSO_2021_6params_NYC_norm_28_PSO_15.csv')
table_2021 = pd.read_csv('data/2_data_for_PSO_calibration/table_2021.csv')

In [5]:
used_cbgs = param_2018['cbg'].tolist()

In [6]:
table_2018 = table_2018[table_2018['A_cbg'].isin(used_cbgs)]
table_2019 = table_2019[table_2019['A_cbg'].isin(used_cbgs)]
table_2020 = table_2020[table_2020['A_cbg'].isin(used_cbgs)]
table_2021 = table_2021[table_2021['A_cbg'].isin(used_cbgs)]

In [7]:
dfs = {'param_2018': param_2018,'param_2019': param_2019,'param_2020': param_2020, 'param_2021': param_2021,
      'table_2018': table_2018,'table_2019': table_2019,'table_2020': table_2020, 'table_2021': table_2021}

In [8]:
vars_definition = {'H_Area_of_store': 'Store area', 
       'R_Percentage_of_Visits_by_brand': 'Chain loyalty', 'J_POI_count_where_store_is': 'POI count',
       'K_POI_diversity_where_store_is': 'POI diversity', 'L_Demographic_similarity': 'Demographic similarity',
       'G_Distance_between_cbg_and_store':'CBG-Store Distance'}

In [9]:
variables = ['H_Area_of_store', 
       'R_Percentage_of_Visits_by_brand', 'J_POI_count_where_store_is',
       'K_POI_diversity_where_store_is', 'L_Demographic_similarity',
       'G_Distance_between_cbg_and_store']

In [10]:
norm = 28

PSO_range = 15

def convert(df, colname):
    list_here = df[colname].tolist()
    max_num = max(list_here)
    min_num = min(list_here)
    deno = max_num - min_num
    ans = []
    for i in list_here:
        this = ((i-min_num)/deno)*(norm-1) + 1
        ans.append(this)
    return ans

In [11]:
def get_table(y):
    t_temp = dfs['table_'+str(y)].copy()
    for i in t_temp.columns:
        if ' ' in i.strip():
            t_temp = t_temp.rename(columns = {i: i.replace(' ','')})
    t_final = t_temp[['A_cbg', 'B_store', 'C_Percentage_of_Visits_'+str(y), 
                      'D_Number_of_Visits_'+str(y)]].rename(columns = {
        'C_Percentage_of_Visits_'+str(y): 'C_Percentage_of_Visits',
        'D_Number_of_Visits_'+str(y): 'D_Number_of_Visits',
        'G_ Distance_between_cbg_and_store': 'G_Distance_between_cbg_and_store'}) 
    change = ['H_Area_of_store', 'R_Percentage_of_Visits_by_brand_'+str(y), 'G_Distance_between_cbg_and_store', 
    'J_POI_count_where_store_is', 'K_POI_diversity_where_store_is',
             'L_Demographic_similarity']
    for i in change:
        t_final[i] = convert(t_temp, i)
        if str(y) in i:
            t_final = t_final.rename(columns = {i:i[:-5]})
    return t_final

In [12]:
def get_predicted_raw_visit_share(row):
    up_first = (row['H_Area_of_store']**row['H_Area_of_store_param'])*(
        row['R_Percentage_of_Visits_by_brand']**row['R_Percentage_of_Visits_by_brand_param'])
    up_second = (row['J_POI_count_where_store_is']**row['J_POI_count_where_store_is_param'])*(
        row['K_POI_diversity_where_store_is']**row['K_POI_diversity_where_store_is_param'])*(
        row['L_Demographic_similarity']**row['L_Demographic_similarity_param'])
    down = row['G_Distance_between_cbg_and_store']**row['G_Distance_between_cbg_and_store_param']
    return up_first*up_second/down

In [13]:
def get_predicted_visits(year):
    df_temp = get_table(year)
    calibrated_params_here = dfs['param_'+str(year)].copy()
    
    for i in ['H_Area_of_store', 
       'R_Percentage_of_Visits_by_brand', 'J_POI_count_where_store_is',
       'K_POI_diversity_where_store_is', 'L_Demographic_similarity',
       'G_Distance_between_cbg_and_store']:
        if i in calibrated_params_here:
            
            calibrated_params_here = calibrated_params_here[calibrated_params_here[i] != 'no visitors']            
            calibrated_params_here[i] = calibrated_params_here[i].astype(float)
            calibrated_params_here = calibrated_params_here.rename(columns = {i:i + '_param'})
    
    computed_df = pd.merge(df_temp, calibrated_params_here.drop(columns = ['cost']), left_on = 'A_cbg', 
                           right_on = 'cbg', how='left').dropna()
    computed_df['result_before_norm'] = computed_df.apply(lambda x: get_predicted_raw_visit_share(x), axis = 1)
    cbg_sum = computed_df.groupby('A_cbg')[['result_before_norm','D_Number_of_Visits']].sum().reset_index().rename(
        columns = {'result_before_norm':'result_cbg_sum', 'D_Number_of_Visits': 'Number_of_Visits_in_this_cbg'})
    computed_df_general = pd.merge(computed_df, cbg_sum, on = 'A_cbg', how = 'left')

    computed_df_general['assumed_visits'] = computed_df_general.apply(
        lambda x: x['result_before_norm']/x['result_cbg_sum']*x['Number_of_Visits_in_this_cbg'], 
        axis = 1)
    return computed_df_general[['A_cbg', 'B_store', 'D_Number_of_Visits', 'assumed_visits']]

In [14]:
years = [2018, 2019, 2020, 2021]

In [15]:
for y in years:
    dfs['preds_'+str(y)] = get_predicted_visits(y)

In [16]:
cbgs = dfs['table_2018']['A_cbg'].unique().tolist()
len(cbgs)

5502

In [17]:
stores = dfs['table_2018']['B_store'].unique().tolist()

In [18]:
ses = ['med_ageE','bachelor_degree_25E', 'med_hh_incomeE', 'whiteE', 'blackE', 'asianE','hispanicE']

In [19]:
store_areas = table_2018[['B_store', 'H_Area_of_store']].drop_duplicates().sort_values(
    by = 'H_Area_of_store').reset_index().drop(columns = 'index')
store_areas

,B_store,H_Area_of_store
0,sg:b200115f9d28436996e8db7376a6acfb,582
1,sg:d07befd8cc324b59b3183aa5dc091015,627
2,sg:7e7476590af643a4b89bdb2fe59464a6,717
3,sg:c843f07483c04a22960814f2d859ec04,805
4,sg:f4ada8c153e14fdc9d019433c2022611,898
...,...,...
277,aggregated_4024_College,214735
278,sg:1b394074dc2e43248068041081ed1b67,214735
279,sg:40c47fece0ba4897b01c7e28efb2731a,472341
280,sg:52de54a8c5b74a3c9f0cd9b44b7830aa,472341


In [20]:
stores_ranked_by_area = store_areas['B_store'].tolist()

In [21]:
def aggregate_cbg_yearly_visits(y):
    return dfs['table_'+str(y)].groupby('A_cbg')['D_Number_of_Visits_'+str(y)].sum().reset_index()
yearly_visits_by_cbg = aggregate_cbg_yearly_visits(2018).merge(aggregate_cbg_yearly_visits(2019)).merge(
    aggregate_cbg_yearly_visits(2020)).merge(aggregate_cbg_yearly_visits(2021))
yearly_visits_by_cbg['visits_min']  = yearly_visits_by_cbg[['D_Number_of_Visits_2018', 'D_Number_of_Visits_2019',
       'D_Number_of_Visits_2020', 'D_Number_of_Visits_2021']].min(axis=1)
yearly_visits_by_cbg

,A_cbg,D_Number_of_Visits_2018,D_Number_of_Visits_2019,D_Number_of_Visits_2020,D_Number_of_Visits_2021,visits_min
0,360050002001,533,596,422,214,214
1,360050002002,932,1045,672,298,298
2,360050002003,413,541,437,149,149
3,360050004001,459,335,278,140,140
4,360050004002,383,392,381,187,187
...,...,...,...,...,...,...
5497,360850319011,1473,1460,1226,795,795
5498,360850319021,987,988,760,291,291
5499,360850319022,1124,1596,1298,625,625
5500,360850319023,972,1245,1039,447,447


In [22]:
#used_cbgs = yearly_visits_by_cbg[yearly_visits_by_cbg['visits_min']>0]['A_cbg'].unique().tolist()

In [23]:
len(used_cbgs)

5502

In [24]:
brands = table_2020[['B_store', 'O_Brand_name']].drop_duplicates()
brands

,B_store,O_Brand_name
0,sg:94b69491e8394408a41b4eaf79700a9a,Family Dollar Stores
1,sg:b261ab19c88f4f68bb8efd01f0bea507,Family Dollar Stores
2,sg:3f8d237767dc4dda8c8ed80f0e7d359d,Family Dollar Stores
3,sg:670f3d81cc5f44f5835954f0285f57af,Family Dollar Stores
4,sg:429f4ee166ff4523be060774d8354a8e,Family Dollar Stores
...,...,...
1518553,sg:287a466b3edd42c18e018c7022c1914f,Kohl's
1518554,sg:2e1dfaeb69074f8cb0cf20ec869a598b,Kohl's
1535058,sg:07fafdb3637c46e88d42a8fc2dce529a,Sears
1535059,sg:9bbf6c92e7cf4768a8cfdccab7bfb693,Sears


In [25]:
print('Pearson correlation by chain')
for y in years:
    d_now = dfs['preds_'+str(y)]
    d_now = d_now[d_now['A_cbg'].isin(used_cbgs)].merge(brands)
    chain_visit_comparison = d_now.groupby(
        ['O_Brand_name'])['assumed_visits','D_Number_of_Visits'].sum().reset_index()
    print(str(y) + ':', scipy.stats.pearsonr(chain_visit_comparison['D_Number_of_Visits'], 
                                             chain_visit_comparison['assumed_visits']))
    print('\n')

Pearson correlation by chain
2018: PearsonRResult(statistic=0.9644761163600984, pvalue=3.323493532266733e-14)


2019: PearsonRResult(statistic=0.9563422059646429, pvalue=3.090861025123563e-13)


2020: PearsonRResult(statistic=0.7423324932114529, pvalue=3.277573575684941e-05)


2021: PearsonRResult(statistic=0.5440493318867697, pvalue=0.005991544194259273)




In [ ]:
table_2019_aggregated = table_2019.merge(
    table_2018[['A_cbg', 'B_store', 'R_Percentage_of_Visits_by_brand_2018']], on = ['A_cbg', 'B_store'])

table_2020_aggregated = table_2020.merge(
    table_2019[['A_cbg', 'B_store', 'R_Percentage_of_Visits_by_brand_2019']], on = ['A_cbg', 'B_store'])

table_2021_aggregated = table_2021.merge(
    table_2020[['A_cbg', 'B_store', 'R_Percentage_of_Visits_by_brand_2020']], on = ['A_cbg', 'B_store'])

In [ ]:
table_2019_aggregated.to_csv(
    'data/table_2019_aggregated.csv', 
    index = False)

table_2020_aggregated.to_csv(
    'data/table_2020_aggregated.csv', 
    index = False)

table_2021_aggregated.to_csv(
    'data/table_2021_aggregated.csv', 
    index = False)